# Quick-Start Guide

**MLRun** is an end-to-end open source MLOps solution to manage and automate your entire analytics and machine learning lifecycle, from data ingestion through model development and full pipeline deployment. MLRun is running as a built-in service in Iguazio and is integrated well with other services in the platform.
Its primary goal is to ease the development of machine learning pipeline at scale and help organization to build a robust process for moving from the research phase to a full operational production.

**Table of Contents**

- [Working with MLRun](#working-with-mlrun)
  - [Train Model](#train-model)
  - [Test Model](#test-model)
  - [Serve Model](#serve-model)

## Working with MLRun

If you need to install MLRun, refer to the [Installation Guide](install.md). Note that if are using the Iguazio platform installed, then the MLRun service is preinstalled. 

Open Jupyter Lab on [**jupyter-lab UI**](http://localhost:30040) and create a new notebook.

### Set Environment

Before you begin, initialize MLRun by calling `set_environment` and provide it the project name

In [1]:
from mlrun import set_environment

project_name = 'quick-start'
_, artifact_path = set_environment(project=project_name)

## Train Model

MLRun introduces the concept of [functions](./runtimes/functions.md). You can run your own code as functions, or use functions from the function marketplace.
In the example below, we'll use the [`sklearn_classifier`](https://github.com/mlrun/functions/tree/master/sklearn_classifier) from the function marketplace to train a model.

In [2]:
from mlrun import import_function
from mlrun.platforms import auto_mount

train = import_function('hub://sklearn_classifier').apply(auto_mount())

train_run = train.run(name='train',
                      inputs={'dataset': 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'},
                      params={'model_pkg_class': 'sklearn.linear_model.LogisticRegression',
                              'label_column': 'label'},
                      project=project_name)

In [3]:
from IPython.display import display, Markdown
import os

#display(Markdown(f'Go to the [**MLRun UI**]({os.environ.get("MLRUN_UI_URL")}/{project_name}/jobs/monitor/{train_run.uid()}) to see the details of this job'))
display(Markdown(f'Go to the [**MLRun UI**]({os.environ.get("MLRUN_UI_URL")}/{project_name}) to see the details of this job'))

<img src="./_static/images/mlrun-quick-start/train-info.png" alt="ui-info" width="800"/>

As well as artifacts:

<img src="./_static/images/mlrun-quick-start/train-artifacts.png" alt="ui-artifacts" width="800"/>

If you run the function in a Jupyter notebook, the output cell for your function execution will contain a table with run information &mdash; including the state of the execution, all inputs and parameters, and the execution results and artifacts.

![MLRun quick start train output](./_static/images/mlrun-quick-start/train-output.png)

## Test Model

Now that you have a trained model, you can test it: run a task that uses the [`test_classifier` marketplace](https://github.com/mlrun/functions/tree/master/test_classifier) function to run the selected trained model against the test data set, as returned for the training task (train) in the previous step.

In [4]:
test = import_function('hub://test_classifier').apply(auto_mount())

You can then run the function as part of your project, just as any other function that you have written yourself. To view the function documentation, call the `doc` method:

In [5]:
test.doc()

function: test-classifier
test a classifier using held-out or new data
default handler: test_classifier
entry points:
  test_classifier: Test one or more classifier models against held-out dataset

Using held-out test features, evaluates the peformance of the estimated model

Can be part of a kubeflow pipeline as a test step that is run post EDA and 
training/validation cycles
    context  - the function context, default=
    models_path(DataItem)  - artifact models representing a file or a folder, default=
    test_set(DataItem)  - test features and labels, default=
    label_column(str)  - column name for ground truth labels, default=
    score_method(str)  - for multiclass classification, default=micro
    plots_dest(str)  - dir for test plots, default=
    model_evaluator  - NOT IMPLEMENTED: specific method to generate eval, passed in as string or available in this folder, default=None
    default_model(str)  - , default=model.pkl
    predictions_column(str)  - column name for the 

Configure parameters for the test function (`params`), and provide the selected trained model from the train task as an input artifact (`inputs`)

In [6]:
test_run = test.run(name="test",
                    params={"label_column": "label"},
                    inputs={"models_path": train_run.outputs['model'],
                            "test_set": train_run.outputs['test_set']},
                    project=project_name)

## Serve Model

MLRun serving can take MLRun models or standard model files and produce managed real-time serverless functions based
on Nuclio real-time serverless engine, which can be deployed everywhere. see [MLRun Serving documentation](./serving/serving-graph.md).

[Nuclio](https://github.com/nuclio/nuclio) is a high-performance "serverless" framework focused on data, I/O, and compute intensive workloads.

To deploy your model using the [`v2_model_server` function](https://github.com/mlrun/functions/tree/master/v2_model_server) run the following code:

In [7]:
serve = import_function('hub://v2_model_server').apply(auto_mount())
model_name='iris'
serve.add_model(model_name, model_path=train_run.outputs['model'])
addr = serve.deploy(project=project_name)

The `invoke` method enables to programmatically test the function.

In [8]:
import json

inputs = [[5.1, 3.5, 1.4, 0.2],
          [7.7, 3.8, 6.7, 2.2]]
my_data = json.dumps({'inputs': inputs})
serve.invoke(f'v2/models/{model_name}/infer', my_data)

In [9]:
display(Markdown(f'Open the [**Nuclio UI**]({os.environ.get("MLRUN_UI_URL")}/projects/{project_name}/functions) to view the function and test it.'))

![MLRun serving UI](./_static/images/mlrun-quick-start/serve.png)

For a more detailed, refer to the [End-to-end Pipeline Tutorial](end-to-end-pipeline.rst).